In [5]:
import pandas as pd
df_part = pd.read_csv(r"F:\celeba\archive\list_eval_partition.csv")

train = df_part[df_part['partition'] == 0]['image_id'].tolist()
val = df_part[df_part['partition'] == 1]['image_id'].tolist()
test = df_part[df_part['partition'] == 2]['image_id'].tolist()

In [7]:
df = pd.read_csv(r"F:\celeba\archive\list_attr_celeba.csv")
df_train = df[df['image_id'].isin(train)]
df_val = df[df['image_id'].isin(val)]
df_test = df[df['image_id'].isin(test)]

In [11]:
df.columns 

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

In [17]:

attrs = ["Bags_Under_Eyes", "Bangs", "Black_Hair", "Blond_Hair", "Brown_Hair", "High_Cheekbones", "Mouth_Slightly_Open", "Chubby", "Eyeglasses", "Gray_Hair", "Narrow_Eyes", "Smiling", "Wearing_Hat"]

df.loc[0, attrs].values

array([-1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, 1, -1], dtype=object)

In [32]:
import torch
# (df.loc[0, attrs].values + 1) // 2
torch.as_tensor((df.loc[0, attrs].values + 1) // 2)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.